# Hyper Tuning a CNN-LSTM Model on Tensorflow Datasets (run on a server)

## Preparation on a server

* create a conda environment with Tensorflow GPU support:
`conda create -n mmproteo tensorflow-gpu python=3.8 jupyter`
    * pay attention that all additional tensorflow packages have (roughly) the same version
* `conda activate mmproteo`
* `pip install -r requirements.txt` (the major `requirements.txt` file from the pride-downloader package, not just the mmproteo one)
* start jupyter server

## Prerequisites

In [1]:
import json
import os
from typing import Callable

import numpy as np
import pandas as pd
import tensorflow as tf
from official.nlp.modeling.layers.position_embedding import RelativePositionEmbedding
import keras_tuner as kt
from mmproteo.utils import log, paths, utils, visualization
from mmproteo.utils.formats.tf_dataset import DatasetLoader
from mmproteo.utils.ml import callbacks, evaluation, layers

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
logger = log.DummyLogger(verbose=True)

INFO: Printing to Stdout


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [5]:
tf.debugging.set_log_device_placement(True)
GPUs = tf.config.list_physical_devices('GPU')
GPUs

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
current_gpu = GPUs[0]
current_gpu

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

## Configuration

In [7]:
%pwd

'/hpi/fs00/home/mirko.krause/masterthesis/pride-downloader/notebooks'

In [8]:
PROJECT = "PXD010000"

In [9]:
DUMP_PATH = "/scratch/mirko.krause/dumps/" + PROJECT
THREAD_COUNT=32

In [10]:
TRAINING_COLUMNS_DUMP_PATH = os.path.join(DUMP_PATH, "training_columns")
FILES_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "*_mzmlid.parquet")
STATISTICS_FILE_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "statistics.parquet")
DATASET_DUMP_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "tf_datasets")
PROCESSING_FILE_PATH = os.path.join(DATASET_DUMP_PATH, "processing_info.json")

In [11]:
SEQ = 'peptide_sequence'

In [12]:
with open(PROCESSING_FILE_PATH, 'r') as file:
    PROCESSING_INFO = json.loads(file.read())
PROCESSING_INFO

{'padding_characters': {'peptide_sequence': '_',
  'mz_array': 0.0,
  'intensity_array': 0.0},
 'padding_lengths': {'mz_array': 2354,
  'intensity_array': 2354,
  'peptide_sequence': 50},
 'idx_to_char': {'0': 'A',
  '1': 'C',
  '2': 'D',
  '3': 'E',
  '4': 'F',
  '5': 'G',
  '6': 'H',
  '7': 'I',
  '8': 'K',
  '9': 'L',
  '10': 'M',
  '11': 'M(Oxidation)',
  '12': 'N',
  '13': 'P',
  '14': 'Q',
  '15': 'R',
  '16': 'S',
  '17': 'T',
  '18': 'V',
  '19': 'W',
  '20': 'Y',
  '21': '_'},
 'normalization': {'intensity_array': '<function base_peak_normalize at 0x7f8e5c0cf550>'},
 'split_value_columns': ['species', 'istrain'],
 'training_data_columns': ['mz_array', 'intensity_array'],
 'target_data_columns': ['peptide_sequence']}

In [13]:
idx_to_char = {int(idx): char for idx, char in PROCESSING_INFO["idx_to_char"].items()}
char_to_idx = {char: idx for idx, char in idx_to_char.items()}

## Loading Tensorflow Datasets

In [14]:
KEEP_CACHE = True

In [15]:
BATCH_SIZE=32

In [16]:
TRAINING_TYPE = 'Train'
TEST_TYPE = 'Test'
EVAL_TYPE = 'Eval'

In [17]:
dataset_file_paths = paths.assign_wildcard_paths_to_splits_grouped_by_path_position_value(
    wildcard_path = os.path.join(
        DATASET_DUMP_PATH, 
        '*',  # filename
        '*',  # species
        '*'   # istrain
    ),
    path_position = -2,
    splits = {
            TRAINING_TYPE: 0.4,
            TEST_TYPE: 0.5,
            EVAL_TYPE: 0.6
        },
    paths_dump_file = os.path.join(
            DATASET_DUMP_PATH,
            "dataset_file_paths.json"
        ),
    skip_existing = KEEP_CACHE,
    logger = logger
)

print()
print("assigned dataset files:")
visualization.print_list_length_in_dict(dataset_file_paths)

INFO: found file paths dump '/scratch/mirko.krause/dumps/PXD010000/training_columns/tf_datasets/dataset_file_paths.json'

assigned dataset files:
#Train = 89
e.g.: /scratch/mirko.krause/dumps/PXD010000/training_columns/tf_datasets/Biodiversity_P_polymyxa_TBS_aerobic_1_17July16_Samwise_16-04-10_mzmlid.parquet/Paenibacillus_polymyxa_ATCC842/Train
#Test = 27
e.g.: /scratch/mirko.krause/dumps/PXD010000/training_columns/tf_datasets/Cj_media_MH_R3_23Feb15_Arwen_14-12-03_mzmlid.parquet/Campylobacter_jejuni/Train
#Eval = 38
e.g.: /scratch/mirko.krause/dumps/PXD010000/training_columns/tf_datasets/M_alcali_copp_CH4_B3_T1_11_QE_23Mar18_Oak_18-01-07_mzmlid.parquet/Methylomicrobium_alcaliphilum/Train


### Loading corresponding TF datasets

In [18]:
element_spec = (
    tuple(tf.TensorSpec(shape=(PROCESSING_INFO['padding_lengths'][col], ), dtype=tf.float32)
     for col in PROCESSING_INFO['training_data_columns']),
    tuple(tf.TensorSpec(shape=(PROCESSING_INFO['padding_lengths'][col], ), dtype=tf.int8)
     for col in PROCESSING_INFO['target_data_columns'])
)
element_spec

((TensorSpec(shape=(2354,), dtype=tf.float32, name=None),
  TensorSpec(shape=(2354,), dtype=tf.float32, name=None)),
 (TensorSpec(shape=(50,), dtype=tf.int8, name=None),))

**In the following step, Tensorflow starts allocating a GPU**

In [19]:
datasets = DatasetLoader(
    element_spec=element_spec,
    batch_size=BATCH_SIZE,
    shuffle_buffer_size=1_000_000,
    thread_count=THREAD_COUNT,
    keep_cache=KEEP_CACHE,
    logger=logger
).load_datasets_by_type(dataset_file_paths)
datasets

DEBUG: preparing dataset 'Train'
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelInterleaveDatasetV4 in device /job:localhost/replica:0/task:0/device:CPU:0
DEBUG: loaded dataset 'Train' interleaved
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
DEBUG: shuffled dataset 'Train'
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
DEBUG: batched dataset 'Train'
DEBUG: cached dataset 'Train'
DEBUG: benchmarked dataset 'Train'
INFO: prepared dataset 'Train'
DEBUG: preparing dataset 'Test'
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelInterleaveDatasetV4 in device /job:localhost/replica:0/task:0/device:CPU:0
DEBUG: loaded dataset 'Test' interleaved
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:

{'Train': <BatchDataset shapes: (((32, 2354), (32, 2354)), ((32, 50),)), types: ((tf.float32, tf.float32), (tf.int8,))>,
 'Test': <BatchDataset shapes: (((32, 2354), (32, 2354)), ((32, 50),)), types: ((tf.float32, tf.float32), (tf.int8,))>,
 'Eval': <BatchDataset shapes: (((32, 2354), (32, 2354)), ((32, 50),)), types: ((tf.float32, tf.float32), (tf.int8,))>}

## Defining the Tensorflow Model

In [20]:
def build_lstm_with_pooling_and_position_model(
    hp: kt.HyperParameters,
    model_name: str = "mmproteo_lstm_with_pooling_and_position"
) -> tf.keras.Model:
    input_layers_list, masked_input_layers_list = layers.create_masked_input_layers(
        [
            layers.InputLayerConfiguration(
                name=col,
                shape=PROCESSING_INFO['padding_lengths'][col],
                mask_value=PROCESSING_INFO['padding_characters'][col]
            )
            for col in PROCESSING_INFO['training_data_columns']
        ]
    )
    
    x = tf.stack(
        values=masked_input_layers_list, 
        axis=-1,
    )

    position_embedding_size = hp.Int('position_embedding_size', min_value=4, max_value=20, step=1)
    
    position_embedding = RelativePositionEmbedding(
        hidden_size=position_embedding_size,
        name='relative_position_embedding'
    )(x)
    position_embedding = tf.expand_dims(position_embedding, 0)
    position_embedding = tf.broadcast_to(
        input=position_embedding, 
        shape=(tf.shape(x)[0], *tf.shape(position_embedding)[1:])
    )
    
    y_layers=[position_embedding]
    
    dense_y = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(
            units=hp.Int('y_time_distributed_dense_units', min_value=4, max_value=64, step=4),
            activation='relu',
            name='y_time_distributed_dense',
        )
    )(x)
    y_layers.append(dense_y)
    
    for i in range(hp.Int('y_number_of_convolutions', min_value=1, max_value=8, step=1)):
        filter_count = hp.Int('y_base_conv_filter_count', min_value=4, max_value=20, step=4) * (i+1)
        kernel_size = hp.Int('y_base_conv_kernel_size', min_value=4, max_value=16, step=2)**i
        cnn_y = tf.keras.layers.Conv1D(
            filters=filter_count,
            kernel_size=kernel_size,
            activation='relu',
            padding='same',
            name=f"y_conv_{kernel_size}_{filter_count}",
        )(x)
        y_layers.append(cnn_y)
    
    x = tf.concat(
        values=y_layers,
        axis=-1
    )
    
    
    y = x
    for i in range(hp.Int('number_of_time_distributed_dense_layers', min_value=0, max_value=4, step=1)):
        y = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(
                units=x.shape[1],
                activation='relu',
                name=f'pre_lstm_time_distributed_dense_{i}',
            )
        )(y)
    
    x = tf.concat(
        values=[x, y],
        axis=-1,
    )
    
    bidirectional_lstm_units_exponent = hp.Int('bidirectional_lstm_units_exponent', min_value=5, max_value=9, step=1)
    x = tf.keras.layers.Bidirectional(
        layer=tf.keras.layers.LSTM(
            units=2**bidirectional_lstm_units_exponent,
            return_sequences=True,
            name='lstm'
        )
    )(x)
    
    x = tf.keras.layers.GlobalMaxPooling1D(
        name='global_max_pooling_over_time',
    )(x)
    
    upscaling_dense_max = hp.Int('upscaling_dense_layer_generator_max', min_value=1, max_value=4, step=1)
    upscaling_dense_min = hp.Int('upscaling_dense_layer_generator_min', min_value=0, max_value=upscaling_dense_max-1, step=1)
    for i in range(upscaling_dense_min, upscaling_dense_max):
        x = tf.keras.layers.Dense(
            units=2**(7 + i),
            activation='relu',
            name=f"upscaling_dense_{i}",
        )(x)
    
    final_dense_feature_units = hp.Int('final_dense_layer_over_length_feature_units', min_value=3, max_value=7, step=1)
    
    x = tf.keras.layers.Dense(
        units=PROCESSING_INFO['padding_lengths'][SEQ] * final_dense_feature_units,
        activation='relu',
        name="final_dense_layer_to_redefine_lengths",
    )(x)
    
    x = tf.reshape(x, (-1, PROCESSING_INFO['padding_lengths'][SEQ], final_dense_feature_units))
    
    x = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(
            units=len(idx_to_char),
            activation=None,
            name='final_time_distributed_dense'
        )
    )(x)
    
    x = tf.keras.activations.softmax(x)
    
    model = tf.keras.Model(
        inputs=input_layers_list, 
        outputs=x, 
        name=f"{model_name}_{utils.get_current_time_str()}")
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
            tf.keras.metrics.SparseCategoricalCrossentropy()
        ]
    )
    
    return model

In [21]:
TUNER_PATH = os.path.join(DUMP_PATH, 'models', 'tuner')
TUNER_PATH

'/scratch/mirko.krause/dumps/PXD010000/models/tuner'

In [22]:
utils.ensure_dir_exists(TUNER_PATH)

In [23]:
tuner = kt.Hyperband(build_lstm_with_pooling_and_position_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=TUNER_PATH,
                     project_name='mmproteo-cnn-lstm')

Invalid model 0/5


Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'
Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'


Invalid model 1/5
Invalid model 2/5


Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'
Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'


Invalid model 3/5
Invalid model 4/5


Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'
Traceback (most recent call last):
  File "/scratch/mirko.krause/anaconda3/envs/mmproteo/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-20-c9ae8c0b4372>", line 24, in build_lstm_with_pooling_and_position_model
    position_embedding = tfnlp.layers.position_embedding.RelativePositionEmbedding(
AttributeError: module 'official.nlp.modeling' has no attribute 'layers'


Invalid model 5/5


RuntimeError: Too many failed attempts to build model.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
TENSORBOARD_LOG_DIR = os.path.join(TUNER_PATH, "tensorboard")
os.path.realpath(TENSORBOARD_LOG_DIR)

In [ ]:
%tensorboard --logdir $TENSORBOARD_LOG_DIR --bind_all

In [ ]:
tuner.search(
    x=datasets[TRAINING_TYPE].repeat(),
    validation_data=datasets[TEST_TYPE].repeat(), 
    validation_steps=500,
    epochs=10,
    steps_per_epoch=int(10_000/BATCH_SIZE),
    callbacks=callbacks.create_callbacks(
        tensorboard=True,
        progressbar=False,
        reduce_lr=True,
        early_stopping=True,
        checkpoints=True,
        csv=True,
        base_path=TUNER_PATH,
    )
)

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
MODEL_PATH = os.path.join(DUMP_PATH, "models", model.name)
MODEL_PATH

In [ ]:
utils.ensure_dir_exists(MODEL_PATH)

In [ ]:
tf.keras.utils.plot_model(
    model=model,
    to_file=os.path.join(MODEL_PATH, "model.png"),
    show_shapes=True
)

In [ ]:
with open(os.path.join(MODEL_PATH, "summary.txt"), 'w') as file:
    def write_lines(line: str) -> None:
        file.write(line)
        file.write("\n")
    model.summary(print_fn=write_lines)
model.summary()

In [ ]:
with open(os.path.join(MODEL_PATH, "model.json"), 'w') as file:
    file.write(model.to_json())

In [ ]:
with open(os.path.join(MODEL_PATH, "model.yaml"), 'w') as file:
    file.write(model.to_yaml())

## Training the Tensorflow Model

In [ ]:
history = model.fit(x=datasets[TRAINING_TYPE].repeat(),
          validation_data=datasets[TEST_TYPE].repeat(), 
          validation_steps=500,
          epochs=1,
          steps_per_epoch=1_000,
          callbacks=callbacks.create_callbacks(
              tensorboard=True,
              progressbar=False,
              reduce_lr=True,
              early_stopping=True,
              checkpoints=True,
              csv=True,
              base_path=MODEL_PATH,
          )
         )
history

## Evaluating the Tensorflow Model

In [ ]:
decode_idx: Callable[[np.ndarray], np.ndarray] = np.vectorize(idx_to_char.get)

In [ ]:
evaluator = evaluation.SequenceEvaluator(
    dataset=datasets[EVAL_TYPE],
    decode_func=decode_idx,
    batch_size=BATCH_SIZE,
    separator=" ",
    padding_character=PROCESSING_INFO['padding_characters'][SEQ],
)

In [ ]:
evaluator.evaluate_model(model)

In [ ]:
eval_df, (x_eval, y_eval, y_pred) = evaluator.evaluate_model_visually(
    model=model,
    sample_size=20,
    keep_separator=True,
)
eval_df

In [ ]:
eval_df.predicted.map(print)
None

broken loss function?

In [ ]:
np.argmax(y_pred[0], axis=1)

In [ ]:
model.predict(datasets[EVAL_TYPE].take(1)).shape

In [ ]:
y_pred.shape

In [ ]:
y_pred